In [ ]:
import pandas as pd
import numpy as np 
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from math import sqrt
from sklearn.model_selection import GridSearchCV
import time 
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import f_classif
from sklearn.metrics import confusion_matrix
import math
from sklearn.metrics import precision_score, make_scorer
import traceback
import io

In [ ]:
import warnings; warnings.simplefilter('ignore')

In [ ]:
inputpath = "/kaggle/input/stockdata"
inputpath

In [ ]:
def pre_process_data(data,null_threshold):
    """
    Drops Date and Unix Date columns from the data.
    Drops the columns which has null values more than specified null_threshold.
    Replaces infinite values with NAN.
    Drops the rows which has null values.

    Parameters
    ----------
    data : dataframe

    null_threshold : numeric
        numeric value describing the amount of null values that can be present.

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    """
    
    data.drop(columns=['Unix Date','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if null_threshold * total / 100 < data[col].isnull().sum():
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data.dropna(axis=0,inplace=True)
    return data

In [ ]:
def dependent_column(data,column):
    """
    Removes all the Next Day columns.
    Removes all the non Growth Rate Columns (GR)
    add the predictor column to list of columns.

    Parameters
    ----------
    data : dataframe

    column : string
        name of the predictor column 

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    column : string
        name of the predictor column
    """
    cols = [col for col in data.columns if "next" not in col.lower() and col.lower().endswith("gr")]
    cols.append(column)
    data = data[cols]
    return (data,column)

In [ ]:
def best_params_logistic(X,Y):
    custom_scorer = make_scorer(metrics.f1_score, greater_is_better=True,pos_label = 1)
    penalty = ['l1', 'l2','none']
    C = [0.001,0.01,0.1,1,10,100]
    hyperparameters = dict(C = C, penalty = penalty)
    logistic = LogisticRegression()
    try:
        gridsearch = GridSearchCV(logistic, hyperparameters, cv=5,verbose=0,scoring = custom_scorer)
        best_model = gridsearch.fit(X, Y)
    except:
        hyperparameters["penalty"] = ['none','l2']
        gridsearch = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0,scoring = custom_scorer)
        best_model = gridsearch.fit(X, Y)
    best_p = best_model.best_estimator_.get_params()["penalty"]
    best_c = best_model.best_estimator_.get_params()["C"]
    return best_p,best_c

In [ ]:
def error_metrics(y_true, y_pred):
    rmse = sqrt(metrics.mean_squared_error(y_true, y_pred))
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    return {"root_mean_squared_error":rmse,"mean_absolute_error":mae,"mean_squared_error":mse}

In [ ]:
def create_confusion_matrix(y_true,y_pred):
    
    cm = confusion_matrix(y_true,y_pred)
    accuracy = metrics.accuracy_score(y_true,y_pred)
    precision = metrics.precision_score(y_true,y_pred)
    recall = metrics.recall_score(y_true,y_pred)
    f1_score = metrics.f1_score(y_true,y_pred)
    return {"accuracy":accuracy,"precision":precision,"recall":recall,"f1_score":f1_score,"confusion matrix":cm}


In [ ]:
def create_logistic_model(df,column, rate,C, penalty):
    df["Target"] = df["Next Day Close Price GR"].apply(lambda x : 1 if x >= rate else 0)
    X = df.drop(columns=["Target",column])
    Y = df["Target"]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
    logmodel = LogisticRegression(penalty = penalty, C = C,random_state = 0)
    logmodel.fit(X_train, y_train)
    y_pred = logmodel.predict(X_test)
    
    result = {}
    error = error_metrics(y_test, y_pred)
    confusion = create_confusion_matrix(y_test,y_pred)
    result.update(error)
    result.update(confusion)
    return result

In [ ]:
def logistic_model(df,column = "Next Day Close Price GR"):
    
    rate_of_growth = [0.001,0.002,0.003,0.004,0.005]
    solution = list()
    for t in rate_of_growth:
        df["Target"] = df["Next Day Close Price GR"].apply(lambda x : 1 if x >= t else 0)
        X = df.drop(columns=["Target",column])
        Y = df["Target"]

        P_values = [0.01,0.05,0.1,0.15,0.2,0.25]
        F_values = [0.1,0.5,1,2,5,10,100]

        f_col_values, p_col_values = f_classif(X,Y)
        f_col_values = dict(zip(X.columns,f_col_values))
        p_col_values = dict(zip(X.columns,p_col_values))
        
        for p in P_values:
            mycols = list()
            for col,val in p_col_values.items():
                if val <= p:
                    mycols.append(col)
            if mycols == []:
                continue
            X = df[mycols]
            X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
            best_p,best_c = best_params_logistic(X_train,y_train)
            result = create_logistic_model(df,column = "Next Day Close Price GR",rate = t,C = best_c, penalty = best_p)
            result.update({"penalty":best_p,"C":best_c,"p_f_value":"p_"+str(p),"rate_of_growth":t})
            result.update({"features":mycols})
            solution.append(result)

        for f in F_values:
            mycols = list()
            for col,val in f_col_values.items():
                if val >= f:
                    mycols.append(col)
            if mycols == []:
                continue
            X = df[mycols]
            X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
            best_p,best_c = best_params_logistic(X_train,y_train)
            result = create_logistic_model(df,column = "Next Day Close Price GR",rate = t,C = best_c, penalty = best_p)
            result.update({"penalty":best_p,"C":best_c,"p_f_value":"f_"+str(f),"rate_of_growth":t})
            result.update({"features":mycols})
            solution.append(result)

    return solution

In [ ]:
security_codes = list()
for filename in os.listdir("../input/newdata/grstocks"):
    security_codes.append(filename[2:-4])
security_codes.sort()

In [ ]:
for name in security_codes:
    try:
        print(name)
        df = pd.read_csv(os.path.join("../input/newdata/grstocks/","gr"+str(name)+".csv"))
        df = pre_process_data(df,60)
        column = "Next Day Close Price GR"
        df,column = dependent_column(df,column)
        result = logistic_model(df,column)
        result_df = pd.DataFrame(result)
        result_df.to_csv("logistic_"+str(name)+".csv",index=None)
    except Exception as e:
        traceback.print_exc() 